In [6]:
import os
import shutil
import xml.etree.ElementTree as ET

In [ ]:
def split_files_by_extension(src_folder, ext_to_targetfolder):
    """
    Moves files from src_folder into target folders according to their extension.
    ext_to_targetfolder: dict, e.g. {'.jpg': 'images', '.txt': 'labels', '.xml': 'annotations'}
    """
    for ext, target_folder in ext_to_targetfolder.items():
        os.makedirs(target_folder, exist_ok=True)

    for fname in os.listdir(src_folder):
        _, ext = os.path.splitext(fname)
        ext = ext.lower()
        if ext in ext_to_targetfolder:
            shutil.copy(
                os.path.join(src_folder, fname),
                os.path.join(ext_to_targetfolder[ext], fname)
            )


In [16]:
def rename_images_and_labels(images_folder, labels_folder, prefix='img_'):
    image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    image_files.sort()  # Ensure predictable order
    idx = 1  # Counter for new names

    for img_file in image_files:
        label_name = os.path.splitext(img_file)[0] + '.txt'
        label_path = os.path.join(labels_folder, label_name)
        if os.path.exists(label_path):
            # Set up new base name
            new_base = f"{prefix}{str(idx).zfill(3)}"
            img_ext = os.path.splitext(img_file)[1]
            new_img_name = f"{new_base}{img_ext}"
            # Rename image
            old_img_path = os.path.join(images_folder, img_file)
            new_img_path = os.path.join(images_folder, new_img_name)
            os.rename(old_img_path, new_img_path)
            # Rename label
            new_label_name = f"{new_base}.txt"
            new_label_path = os.path.join(labels_folder, new_label_name)
            os.rename(label_path, new_label_path)
            idx += 1
        else:
            print(f"Skipped: {img_file} (no matching label)")


In [14]:
images_folder = 'dataset/input/images'
labels_folder = 'dataset/input/labels'

rename_images_and_labels(images_folder, labels_folder, prefix='img_')

In [5]:
class_mapping = {
    0: 'apple',
    1: 'avocado',
    2: 'banana',
    3: 'kiwi',
    4: 'lemon',
    5: 'orange',
    6: 'pear',
    7: 'pomegranate',
    8: 'strawberry',
    9: 'watermelon'
}

In [8]:
def voc_to_yolo(xml_file, class_map, output_txt):
    """ Convert VOC XML annotation to YOLO format text file."""
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    yolo_lines = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        class_idx = class_map.get(name)
        if class_idx is None:
            continue  # Skip if not in map
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)
        
        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h
        
        yolo_line = f"{class_idx} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n"
        yolo_lines.append(yolo_line)

    with open(output_txt, 'w') as f:
        f.writelines(yolo_lines)

def batch_convert_voc_to_yolo(input_folder, output_folder, class_map):
    os.makedirs(output_folder, exist_ok=True)
    for fname in os.listdir(input_folder):
        if fname.lower().endswith('.xml'):
            xml_path = os.path.join(input_folder, fname)
            base = os.path.splitext(fname)[0]
            txt_path = os.path.join(output_folder, base + '.txt')
            voc_to_yolo(xml_path, class_map, txt_path)

In [9]:
# Add more image samples and labels to the dataset
split_files_by_extension(
    'dataset/fruit_images (apple, banana, orange)',
    {
        '.jpg': images_folder,
        '.jpeg': images_folder,
        '.png': images_folder,
        '.txt': labels_folder,
        '.xml': labels_folder
    }
)

In [11]:
def remove_verified_xml(folder):
    for fname in os.listdir(folder):
        if fname.lower().endswith('.xml'):
            base = os.path.splitext(fname)[0]
            txt_name = base + '.txt'
            txt_path = os.path.join(folder, txt_name)
            if os.path.exists(txt_path):
                xml_path = os.path.join(folder, fname)
                os.remove(xml_path)
                print(f"Deleted: {xml_path}")
            else:
                print(f"Skipped (no TXT): {fname}")

In [13]:
# Convert all VOC XML annotations to YOLO format, and remove XMLs with verified TXT
batch_convert_voc_to_yolo(
    labels_folder,
    labels_folder,
    class_mapping
)
remove_verified_xml(labels_folder)

Deleted: dataset/input/labels/apple_1.xml
Deleted: dataset/input/labels/apple_10.xml
Deleted: dataset/input/labels/apple_11.xml
Deleted: dataset/input/labels/apple_12.xml
Deleted: dataset/input/labels/apple_13.xml
Deleted: dataset/input/labels/apple_14.xml
Deleted: dataset/input/labels/apple_15.xml
Deleted: dataset/input/labels/apple_16.xml
Deleted: dataset/input/labels/apple_17.xml
Deleted: dataset/input/labels/apple_18.xml
Deleted: dataset/input/labels/apple_19.xml
Deleted: dataset/input/labels/apple_2.xml
Deleted: dataset/input/labels/apple_20.xml
Deleted: dataset/input/labels/apple_21.xml
Deleted: dataset/input/labels/apple_22.xml
Deleted: dataset/input/labels/apple_23.xml
Deleted: dataset/input/labels/apple_24.xml
Deleted: dataset/input/labels/apple_25.xml
Deleted: dataset/input/labels/apple_26.xml
Deleted: dataset/input/labels/apple_27.xml
Deleted: dataset/input/labels/apple_28.xml
Deleted: dataset/input/labels/apple_29.xml
Deleted: dataset/input/labels/apple_3.xml
Deleted: datas